In [30]:
#Import libraries
import numpy as np
import pandas as pd

In [106]:
###Transformed Data filtered by gender
##Function to generate working data for Gender Men/Women
def transform_data(data,gender):
    #Filter Gender as Men from dump
    dump_data=dump_data[dump_data["Gender"]=="Men"]

    #Filter required columns
    data1=dump_data.drop(['Client_Category','Qty','Tax','Retail_Price','Mem_Disc','Oth_Disc'],axis=1)

    #Modify date
    data1['Created_Date2'] = pd.to_datetime(data1['Created_Date2'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
    data1=data1.sort_values(by="Created_Date2")

    #Derive frequency of clients as featured column
    data1['Frequency']=data1.groupby("ClientID")["Created_Date2"].transform('nunique')
    #Derive total lifetime spendings of clients as featured column
    data1['Spendings']=data1.groupby("ClientID")["Total"].transform('sum')

    #Function to categorise clients based on frequency &  spendings
    def f(row):
        if row['Frequency'] >= 9:
            val = 'Loyal'
        elif row['Frequency'] == 1:
            val = 'Walkin'
        elif row['Frequency'] > 4 and  row['Spendings'] >= 50000:
            val = 'Premium'
        elif row['Frequency'] > 4 and  row['Spendings'] >= 35000:
            val = 'VIP'
        elif row['Frequency'] >=2 :
            val='Repeat'
        else:
            val="others"    
        return val

    #Derive category of clients based on frequency &  spendings as featured column
    data1['Client_Category'] = data1.apply(f, axis=1)
    return data1

###New-Returning Total clients
##Function to New-Returning Total clients
def new_returning_totalclients(data):
    #Number of unique clients
    freqdata1=data.groupby(["ClientID","Service_Category","EmpID"]).size().reset_index(name="Frequency")
    freqdata2 = freqdata1.groupby(['EmpID','Service_Category']).size().reset_index(name="clients")
    #freqdata2[freqdata2["EmpID"]==500358]

    #Copy data1 to df1
    df1=data.copy()

    #New and returning clients
    df1['date_of_first_engagement']=df1.groupby(["ClientID","EmpID"])["Created_Date2"].transform("first")

    def f1(row):
        if row['Created_Date2'] == row['date_of_first_engagement']:
            val = 'new'
        elif row['Created_Date2'] > row['date_of_first_engagement']:
            val = 'returning'
        return val

    df1['client_status'] = df1.apply(f1, axis=1)

    data12=df1.groupby(["ClientID",'client_status',"EmpID","Service_Category"]).size().reset_index(name="client_count")
    data12 = data12.groupby(["EmpID","Service_Category","client_status"]).size().reset_index()
    data12 = pd.pivot_table(data12,index=['EmpID','Service_Category'],columns='client_status',fill_value=0).reset_index()

    #Merge total client count with new & returning client count 
    final_data_nr = pd.merge(data12,freqdata2,on=['EmpID','Service_Category'],how='left')
    #final_data_nr[final_data_nr["EmpID"]==500358]
    final_data_nr = final_data1.reindex(columns=['EmpID','Service_Category','Total_clients','new','returning'])
    return final_data_nr


###Clients lost to salon(LCS)
##Function to Clients lost to salon(LCS)
def LCS(data):
    #Copy data1 to data2
    data2=data.copy()

    #Modify Date
    data2['Created_Date2'] = data2['Created_Date2'].astype('datetime64[ns]')

    #Filter required columns
    sal_lost=data2[["SalonID","ClientID","Created_Date2","Service_Category"]]
    #sal_lost[sal_lost['ClientID']=='MOHIDEEN9442206300'].head()
    sal_lost=sal_lost.drop_duplicates()
    sal_lost['diff'] = sal_lost.groupby(['ClientID','Service_Category'])['Created_Date2'].diff() / np.timedelta64(1, 'D')
    sal_lost['diff'] = sal_lost['diff'].fillna(0)

    def f2(row):
        if (row['diff'] <= 45):
            val = 'retained'
        else:
            val = 'Lost'
        return val

    sal_lost['diff_status'] = sal_lost.apply(f2,axis=1)

    sal_lost1=sal_lost.groupby(["SalonID","ClientID","Service_Category","diff_status"]).size().reset_index(name="ret_count")
    sal_lost2=pd.pivot_table(sal_lost1,index=['SalonID','ClientID','Service_Category'],columns='diff_status',fill_value=0).reset_index()
    #sal_lost2[sal_lost2["ClientID"]=='MOHIDEEN9442206300']
    sal_lost2.columns=['SalonID','ClientID','Service_Category','Lost_count_salon','retained_count']
    sal_lost2["total_count"]=sal_lost2["Lost_count_salon"]+sal_lost2["retained_count"]
    sal_lost3=sal_lost2.groupby(["SalonID","Service_Category"])["Lost_count_salon"].sum().reset_index()
    #sal_lost3[sal_lost3["SalonID"]==101].head(3)
    return sal_lost3


##Clients lost to employee(LCE)
##Function to Clients lost to employee(LCE)
def LCE(data):
    #Copy data1 to data2
    data2=data.copy()

    #Modify Date
    data2['Created_Date2'] = data2['Created_Date2'].astype('datetime64[ns]')

    emp_lost=data2[["SalonID","EmpID","ClientID","Service_Category","Created_Date2"]]
    #emp_lost[emp_lost['ClientID']=='MOHIDEEN9442206300'].head(2)
    emp_lost=emp_lost.drop_duplicates()
    emp_lost['diff'] = emp_lost.groupby(["EmpID",'ClientID','Service_Category'])['Created_Date2'].diff() / np.timedelta64(1, 'D')
    emp_lost['diff'] = emp_lost['diff'].fillna(0)

    def f3(row):
        if (row['diff'] <= 45):
            val = 'retained'
        else:
            val = 'Lost'
        return val

    emp_lost['diff_status'] = emp_lost.apply(f3,axis=1)

    emp_lost1=emp_lost.groupby(["SalonID","EmpID","ClientID","Service_Category","diff_status"]).size().reset_index(name="ret_count")
    emp_lost2=pd.pivot_table(emp_lost1,index=['SalonID','EmpID','ClientID','Service_Category'],columns='diff_status',fill_value=0).reset_index()
    emp_lost2.columns=['SalonID','EmpID','ClientID','Service_Category','Lost_count_Emp','retained_count']
    #emp_lost2[emp_lost2["ClientID"]=='MOHIDEEN9442206300']
    emp_lost2["total_count"]=emp_lost2["Lost_count_Emp"]+emp_lost2["retained_count"]
    emp_lost3=emp_lost2.groupby(["SalonID","EmpID","Service_Category"])["Lost_count_Emp"].sum().reset_index()
    #emp_lost3[emp_lost3["EmpID"]==701317]
    return emp_lost3


###Merge LCS & LCE to New-Returning-Total clients
##Function to create final table for employee recommendation
def emp_recomm(data_nrt,data_lce,data_lcs):
    #Merge LCE
    final_data=pd.merge(data_lce,data_nrt,on=("EmpID","Service_Category"),how='left')
    final_data.columns=['SalonID','EmpID','Service_Category','Lost_count_Emp','Total_clients','new','returning']
    #final_data.head(3)

    #Compute weighted rating using Returning client & LCE count
    final_data['Ret_rating'] = (0.4*final_data['Lost_count_Emp']+0.6*final_data['returning'])/100
    final_data=final_data.sort_values(by="Ret_rating",ascending=False)
    #final_data[final_data['EmpID']==500358]

    #Merge LCS
    final_data1=pd.merge(final_data,data_lcs,on=("SalonID","Service_Category"),how='left')

    final_data2 = final_data1.reindex(columns=['SalonID','EmpID','Service_Category','Total_clients','new','returning','Lost_count_salon','Lost_count_Emp','Ret_rating'])
    #final_data2[(final_data2["SalonID"]==198) & (final_data2["Service_Category"]=="STYLING")] 
    return final_data2

In [ ]:
##Load data
dump_data=pd.read_csv("datadump_cleaned.csv")

##Transformed Data filtered by gender
data1 = transform_data(dump_data,"Men")
#data1 = transform_data(dump_data1,"Women")

##New-Returning Total clients
data_nrt = new_returning_totalclients(data1)
##Clients lost to salon(LCS)
data_lcs = LCS(data1)
##Clients lost to employee(LCE)
data_lce = LCE(data1)

##Employee recommendation
data_emp_recomm = emp_recomm(data_nrt,data_lce,data_lcs)
data_emp_recomm.head()